In [149]:
from IPython.display import display
import altair as alt

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [150]:
import duckdb

# Le dataset
csv = '../../data/toflit18_all_flows.csv'

# Lire le csv et l'envoyer comme une relation (table)
toflit_flows = duckdb.sql(f"""
SELECT  customs_region,
        CAST(value AS FLOAT) as value,
        year,
        export_import,
        best_guess_region_prodxpart
        best_guess_national_prodxpart,
        best_guess_national_partner,
        best_guess_national_product,
        best_guess_region_prodxpart,
        best_guess_national_region,
        origin_province,
        partner_grouping,
        partner_simplification
FROM read_csv_auto('{csv}', ALL_VARCHAR=TRUE)
""")

In [151]:
# Appliquer les filtres de base (communes aux tous buckets)

base_relation = duckdb.sql(f"""
    FROM toflit_flows
    WHERE customs_region = 'Marseille'
    AND year = '1789'
    AND value IS NOT NULL
    AND export_import = 'Exports'
    AND best_guess_region_prodxpart = '1'
""")

base_relation.show()

┌────────────────┬──────────┬─────────┬───────────────┬───┬─────────────────┬──────────────────┬──────────────────────┐
│ customs_region │  value   │  year   │ export_import │ … │ origin_province │ partner_grouping │ partner_simplifica…  │
│    varchar     │  float   │ varchar │    varchar    │   │     varchar     │     varchar      │       varchar        │
├────────────────┼──────────┼─────────┼───────────────┼───┼─────────────────┼──────────────────┼──────────────────────┤
│ Marseille      │   2312.5 │ 1789    │ Exports       │ … │ Provence        │ France           │ Marseille            │
│ Marseille      │   4237.0 │ 1789    │ Exports       │ … │ NULL            │ France           │ Marseille            │
│ Marseille      │  43636.5 │ 1789    │ Exports       │ … │ Bourgogne       │ France           │ Marseille            │
│ Marseille      │   1593.0 │ 1789    │ Exports       │ … │ Dauphiné        │ France           │ Marseille            │
│ Marseille      │ 194160.0 │ 1789    │ 

In [152]:
# BUCKET 1 (Marseille) - partner_grouping = 'France' AND partner_simplification = 'Marseille'

marseille_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping = 'France'
AND partner_simplification = 'Marseille'
""")

marseille_bucket.show()

┌────────────────┬───────────┬─────────┬───────────────┬───┬─────────────────┬──────────────────┬──────────────────────┐
│ customs_region │   value   │  year   │ export_import │ … │ origin_province │ partner_grouping │ partner_simplifica…  │
│    varchar     │   float   │ varchar │    varchar    │   │     varchar     │     varchar      │       varchar        │
├────────────────┼───────────┼─────────┼───────────────┼───┼─────────────────┼──────────────────┼──────────────────────┤
│ Marseille      │    2312.5 │ 1789    │ Exports       │ … │ Provence        │ France           │ Marseille            │
│ Marseille      │    4237.0 │ 1789    │ Exports       │ … │ NULL            │ France           │ Marseille            │
│ Marseille      │   43636.5 │ 1789    │ Exports       │ … │ Bourgogne       │ France           │ Marseille            │
│ Marseille      │    1593.0 │ 1789    │ Exports       │ … │ Dauphiné        │ France           │ Marseille            │
│ Marseille      │    2586.0 │ 1

In [153]:
# BUCKET 2 (France) - partner_grouping = 'France' AND partner_simplification != 'Marseille'

france_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping = 'France'
AND partner_simplification != 'Marseille'
""")

france_bucket.show()

┌────────────────┬───────────┬─────────┬───────────────┬───┬─────────────────┬──────────────────┬──────────────────────┐
│ customs_region │   value   │  year   │ export_import │ … │ origin_province │ partner_grouping │ partner_simplifica…  │
│    varchar     │   float   │ varchar │    varchar    │   │     varchar     │     varchar      │       varchar        │
├────────────────┼───────────┼─────────┼───────────────┼───┼─────────────────┼──────────────────┼──────────────────────┤
│ Marseille      │ 26325.771 │ 1789    │ Exports       │ … │ NULL            │ France           │ Dunkerque            │
│ Marseille      │   2198.44 │ 1789    │ Exports       │ … │ NULL            │ France           │ Bayonne              │
│ Marseille      │   3015.46 │ 1789    │ Exports       │ … │ NULL            │ France           │ Dunkerque            │
│ Marseille      │     116.0 │ 1789    │ Exports       │ … │ NULL            │ France           │ Corse                │
│ Marseille      │     154.0 │ 1

In [161]:
# BUCKET 3 (monde) - partner_grouping != 'France'

monde_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping != 'France'
""")

monde_bucket.show()

┌────────────────┬────────────┬─────────┬───┬─────────────────┬──────────────────┬──────────────────────┐
│ customs_region │   value    │  year   │ … │ origin_province │ partner_grouping │ partner_simplifica…  │
│    varchar     │   float    │ varchar │   │     varchar     │     varchar      │       varchar        │
├────────────────┼────────────┼─────────┼───┼─────────────────┼──────────────────┼──────────────────────┤
│ Marseille      │   194160.0 │ 1789    │ … │ Amérique        │ Monde            │ Monde hors colonies  │
│ Marseille      │     9048.0 │ 1789    │ … │ Monde           │ Angleterre       │ Angleterre           │
│ Marseille      │ 10334644.0 │ 1789    │ … │ Amérique        │ Monde            │ Monde hors colonies  │
│ Marseille      │    1270.25 │ 1789    │ … │ Provence        │ Angleterre       │ Angleterre           │
│ Marseille      │     2625.0 │ 1789    │ … │ Amérique        │ Monde            │ Monde hors colonies  │
│ Marseille      │     6616.0 │ 1789    │ … │ 

In [155]:
def group_by_province(relation) -> list[dict]:
    aggreg = relation.aggregate("""
        origin_province,
        SUM(value) as total_value
    """)

    values = aggreg.fetchall()
    columns = aggreg.columns

    return [dict(zip(columns, row)) for row in values]

In [156]:
def visualise(aggregations, title):
    
    return alt.Chart().from_dict({
        "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
        "title": title,
        "mark": "bar",
        "data": {
            "values": aggregations
        },
        "encoding": {
            "x": {
                "field": "total_value",
                "type": "quantitative", 
                "title": "value"
            },
            "y": {
                "field": "origin_province",
                "type": "nominal",
                "title": "origin_province",
                "sort": "-x"
            }
        }
    })

In [157]:
marseille_bucket_data = group_by_province(marseille_bucket)
marseille_bucket_data

[{'origin_province': 'Provence', 'total_value': 8127777.450965881},
 {'origin_province': None, 'total_value': 42897.0},
 {'origin_province': 'Bourgogne', 'total_value': 65461.25},
 {'origin_province': 'Dauphiné', 'total_value': 161474.73046875},
 {'origin_province': 'Languedoc', 'total_value': 12452093.75},
 {'origin_province': 'Lyonnais', 'total_value': 7223073.910186768},
 {'origin_province': 'Île-de-France', 'total_value': 569400.0},
 {'origin_province': 'Normandie', 'total_value': 1256823.25},
 {'origin_province': 'Monde', 'total_value': 235966.8857421875},
 {'origin_province': 'Inconnu', 'total_value': 118739.0},
 {'origin_province': 'Orléanais', 'total_value': 361464.0},
 {'origin_province': 'Picardie', 'total_value': 499890.5},
 {'origin_province': 'Champagne', 'total_value': 1326674.25},
 {'origin_province': 'Angleterre', 'total_value': 35733.0},
 {'origin_province': 'Flandre', 'total_value': 1054792.4497070312},
 {'origin_province': 'Maine', 'total_value': 253363.0},
 {'origin

In [ ]:
chart = visualise(marseille_bucket_data, 'Origine des produits exportés depuis Marseille vers Marseille')
chart

In [158]:
france_bucket_data = group_by_province(france_bucket)
france_bucket_data

[{'origin_province': None, 'total_value': 1350036.4332504272}]

In [ ]:
chart = visualise(france_bucket_data, 'Origine des produits exportés depuis Marseille vers partenaires en France')
chart

In [159]:
monde_bucket_data = group_by_province(monde_bucket)
monde_bucket_data

[{'origin_province': 'Amérique', 'total_value': 24912626.0},
 {'origin_province': 'Monde', 'total_value': 719270.0},
 {'origin_province': 'Provence', 'total_value': 8983025.08984375},
 {'origin_province': None, 'total_value': 42228648.05075073},
 {'origin_province': 'Lyonnais', 'total_value': 1538081.0},
 {'origin_province': 'Île-de-France', 'total_value': 530604.0},
 {'origin_province': 'Dauphiné', 'total_value': 153492.0},
 {'origin_province': 'Languedoc', 'total_value': 749237.0},
 {'origin_province': 'Levant et Barbarie', 'total_value': 128257.0},
 {'origin_province': 'Flandre', 'total_value': 120619.0},
 {'origin_province': 'Inconnu', 'total_value': 148001.0},
 {'origin_province': 'Normandie', 'total_value': 25460.0},
 {'origin_province': 'Champagne', 'total_value': 30153.0},
 {'origin_province': 'Asie', 'total_value': 1578281.0},
 {'origin_province': 'Auvergne', 'total_value': 88244.0},
 {'origin_province': 'Picardie', 'total_value': 143473.0},
 {'origin_province': 'Touraine', 't

In [ ]:
chart = visualise(monde_bucket_data, "Origine des produits exportés effectués depuis Marseille vers l'étranger")
chart

In [160]:
# Voir qui sont les partenaires de la marchandise 
# exportée depuis Marseille vers la France 

france_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping = 'France'
AND partner_simplification != 'Marseille'
""")

def group_by_partner(relation) -> list[dict]:
    aggreg = relation.aggregate("""
        partner_simplification,
        SUM(value) as total_value,
        COUNT(value) as nb_value,
    """)

    values = aggreg.fetchall()
    columns = aggreg.columns

    return [dict(zip(columns, row)) for row in values]

france_origin_is_null_grouped_by_partner = group_by_partner(france_bucket)

france_origin_is_null_grouped_by_partner

[{'partner_simplification': 'Dunkerque',
  'total_value': 655032.4423675537,
  'nb_value': 43},
 {'partner_simplification': 'Bayonne',
  'total_value': 280171.2106628418,
  'nb_value': 25},
 {'partner_simplification': 'Corse',
  'total_value': 350871.0802078247,
  'nb_value': 48},
 {'partner_simplification': 'Lorient',
  'total_value': 63961.70001220703,
  'nb_value': 4}]